In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

import requests
import json
from datetime import timedelta

from geopy import Nominatim

In [54]:
df = pd.read_csv('/content/drive/MyDrive/usgs_data.csv',
                 engine='python')

In [55]:
df.head()

,Unnamed: 0,p_name,manufacturer_processed,longitude,latitude,numb_turbines,avg_turbine_capacity,turbine_blade_weight,turbine_blade_length,text
0,49,unknown Wyoming County,other,-78.187935,42.740818,1.0,NaN,0.66904,41.0,Project Name: unknown Wyoming County<br>Number...
1,48,unknown Steuben County,other,-77.533662,42.486564,11.0,NaN,0.66904,41.0,Project Name: unknown Steuben County<br>Number...
2,47,unknown Oneida County,other,-75.447369,42.975771,2.0,NaN,0.66904,41.0,Project Name: unknown Oneida County<br>Number ...
3,46,unknown Niagara County,other,-78.493963,43.282899,1.0,NaN,0.66904,41.0,Project Name: unknown Niagara County<br>Number...
4,45,unknown Montgomery County,other,-74.510246,42.905187,2.0,NaN,0.66904,41.0,Project Name: unknown Montgomery County<br>Num...


In [56]:
def TravelInfo(origin, destination):
  """
  orgin: (x,y) where x and y represent latitude and longitude respectively
  destination: (x,y) where x and y represent latitude and longitude respectively

  returns travel distance in mi
  """
  # Unpack the latitudes and longitudes
  (lat1, lon1) = origin
  (lat2, lon2) = destination
  # Make API call to OSRM (Open Source Routing Machine) for travel info
  r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{lon1},{lat1};{lon2},{lat2}?overview=false""")
  # Unpack desired info
  route_1 = json.loads(r.content)["routes"][0]
  distance = route_1['distance']
  duration = route_1['duration']
  return distance/1609

In [68]:
def PredictTransportEmission(Windturbine,Destination):
  """
  Predicts GHG emissions for transporting one windmill blade

  destination: (x,y) where x and y represent latitude and longitude respectively

  Windturbine is a row from the df with windmill info.
  ex: Windturbine = df.iloc[7]
  """
  # Assume flatbed is 16.2 meters, we need __ flatbeds for transport
  numb_trucks = np.ceil(Windturbine['turbine_blade_length']/16.2)
  # Transport Distance (mi)
  Origin = (Windturbine['latitude'],Windturbine['longitude'])
  transport_distance = TravelInfo(Origin, Destination)
  # Shipment weight (lbs) --------------------------------------------------------- This we can create a model for if we can find some data
  weight_tons = Windturbine['turbine_blade_weight']       # estimation based on XGBoost model
  # Total # of ton miles
  ton_miles = weight_tons * transport_distance
  # Trucks emit 161.8 grams per ton-mile
  emission_factor = 161.8

  ghg_emissions = transport_distance * weight_tons * emission_factor
  print(f'We expect {ghg_emissions} grams of GHG emissions.')
  print(f'That is {ghg_emissions/1000} kilograms of GHG emissions;')
  print(f'or, {ghg_emissions/1000000000} kilotons. ')


In [69]:
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('Delaware Park, Buffalo, NY')

In [70]:
Windmill = df.iloc[10]
Windmill

Unnamed: 0                                                               39
p_name                                            unknown Chautauqua County
manufacturer_processed                                                other
longitude                                                         -79.11637
latitude                                                           42.55933
numb_turbines                                                           1.0
avg_turbine_capacity                                                    NaN
turbine_blade_weight                                                0.66904
turbine_blade_length                                                   41.0
text                      Project Name: unknown Chautauqua County<br>Num...
Name: 10, dtype: object

In [71]:
PredictTransportEmission(
    Windmill,
    (location.latitude,location.longitude)
                         )

We expect 4163.075330116762 grams of GHG emissions.
That is 4.1630753301167625 kilograms of GHG emissions;
or, 4.163075330116763e-06 kilotons. 
